# Interview experiments for blenderbot

In order to get Freja to act as an inteviewer we have to constrain her. Blenderbot(the model) is trained as an open-domain chat bot, which makes her very explorative when it comes to asking questions.

#### Approach:
- Hard code a set of interview questions
- System for determining when it's time to move on to next question
- Experiment with the dialogue context she gets at each pass of the conversation


## BlenderConversation is a class for storing the conversation with blenderbot

In [1]:
class BlenderConversation:
    
    def __init__(self,lang,tokenizer,description='No description'):
        self.lang = lang
        self.description = description
        self.bot_text = []
        self.user_text = []
        self.user_turn = True
        self.tokenizer = tokenizer
        
    def reset(self):
        self.bot_text = []
        self.user_text = []
        self.user_turn = True
        
    def add_user_text(self,text):
        if self.user_turn:
            self.user_text.append(text)
            self.user_turn = False
        else:
            raise ValueError("It's the bot's turn to add a reply to the conversation")
        return
    
    def add_bot_text(self,text):
        if not self.user_turn:
            self.bot_text.append(text)
            self.user_turn = True
        else:
            raise ValueError("It's the user's turn to add an input to the conversation")
        return
    
    def pop(self):
        if self.user_turn:
            self.bot_text.pop()
            self.user_turn = False
        else:
            self.user_text.pop()
            self.user_turn = True
        return
    
    def get_bot_replies():
        # TODO: Option to return string instead of list?
        return self.bot_text
    
    def get_user_replies():
        # TODO: Option to return string instead of list?
        return self.user_text
        
    def get_dialogue_history(self,max_len=100):
        # Returns string of the dialogue history with bot and user inputs separated with '\n'
        # max_len set to default 110 as model has max input length 128 and we want some space for new input 
        history = ''
        tokens_left = max_len
        if self.user_turn:
            # Start backwards from bot_text
            for i in reversed(range(len(self.user_text))):
                bot_text = self.bot_text[i]
                user_text = self.user_text[i]
                nbr_tokens = len(self.tokenizer(bot_text)['input_ids'])  + len(self.tokenizer(user_text)['input_ids'])
                if  nbr_tokens < tokens_left: # This is not fool proof as the model tokenizer tokenizes differently
                    history = user_text + '\n' + bot_text + '\n' + history
                    tokens_left -= (nbr_tokens + 2)
                else:
                    break
                                
        else:
            # Start backwards from user_text
            history = self.user_text[-1]
            tokens_left -= len(self.tokenizer(history)['input_ids'])
            for i in reversed(range(len(self.user_text)-1)):
                bot_text = self.bot_text[i]
                user_text = self.user_text[i]
                nbr_tokens = len(bot_text.split()) + len(user_text.split())
                if  nbr_tokens < tokens_left: # This is not fool proof as the model tokenizer tokenizes differently
                    history = user_text + '\n' + bot_text + '\n' + history
                    tokens_left -= (nbr_tokens + 2)
                else:
                    break
        return history
        
    def to_txt(self,file='None'):
        # Writes the dialogue to txt file in subdirectory
        text = '####################################\n' + 'Conversation description: ' + self.description + '\n\n'
        if self.user_turn:
            for i in range(len(self.user_text)):
                text = text + 'User>>> '+ self.user_text[i] + '\n Bot>>> ' + self.bot_text[i] + '\n'
        else:
            for i in range(len(self.bot_text)):
                text = text + 'User>>> '+ self.user_text[i] + '\n Bot>>> ' + self.bot_text[i] + '\n'
            text = text + 'User>>> ' + self.user_text[-1]
        
        if file is None:
            if self.lang == 'sv':
                file = 'interview_sv.txt'
            else:
                file = 'interview_en.txt'
        
        text = text + '\n\n'
        file_path = '02_interview_output/' + file
        with open(file_path,'a') as f:
            f.write(text)
        return
         
    
    def print_dialogue(self):
        # Prints the dialogue 
        text = ''
        if self.user_turn:
            for i in range(len(self.user_text)):
                text = text + 'User>>> '+ self.user_text[i] + '\n Bot>>> ' + self.bot_text[i] + '\n'
        else:
            for i in range(len(self.bot_text)):
                text = text + 'User>>> '+ self.user_text[i] + '\n Bot>>> ' + self.bot_text[i] + '\n'
            text = text + 'User>>> ' + self.user_text[-1]
        print(text)
        return


def strip_token(line):
    # Removes SOS and EOS tokens from blenderbot reply
    line = line.replace('<s>','')
    line = line.replace('</s>','')
    return line


## Class for keeping track of the interview and 

In [2]:
# Just a place to store questions instead of having txt files for now
import random
questions = ['Du har sökt jobbet som {}. Vad är det som du tycker verkar vara roligt med detta arbetet?', 
                       'Om du ska arbeta som {} så är det bra om du har erfarenhet från YY. Kan du berätta lite om du har sådan erfarenhet?',
                       'Vad är din bästa erfarenhet från dina tidigare arbeten?',
                       'Vad gör att du skulle passa bra som {}?',
             'Är det någonting som du vill fråga om detta arbetet?']
format_question = [1,1,0,1,0]
interview_questions= zip(questions,format_question)

['Om du ska arbeta som car mech så är det bra om du har erfarenhet från YY. Kan du berätta lite om du har sådan erfarenhet?',
 'Du har sökt jobbet som car mech. Vad är det som du tycker verkar vara roligt med detta arbetet?',
 'Vad är din bästa erfarenhet från dina tidigare arbeten?',
 'Är det någonting som du vill fråga om detta arbetet?',
 'Vad gör att du skulle passa bra som car mech?']

In [17]:
from googletrans import Translator
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from torch import no_grad
import random
from pathlib import Path
from IPython.core.debugger import set_trace

class InterviewWorld:
    # Class that keeps

    def __init__(self, job, name, mname='facebook/blenderbot-1B-distill'):
        # TODO: More sophisticated questions/greeting drawn from txt file(?) and formated with name and job
        # TODO: init model and tokenizer from file
        # TODO: init from opt like dictionary
        self.questions = [question.format(job) if format_this else question for (question, format_this) in
                          read_questions('interview_questions.txt')]
        random.shuffle(self.questions)
        self.greeting = 'Hej, och välkommen till din intervju. Hur står det till, {}?'.format(name)
        self.context = ''  # TODO, maybe a function that updates this as well

        self.job = job
        self.human_name = name
        self.model = BlenderbotForConditionalGeneration.from_pretrained(mname)  # Try to load on gpu
        self.tokenizer = BlenderbotTokenizer.from_pretrained(mname)
        self.model_name = mname.replace('facebook/', '')
        self.translator = Translator()
        self.episode_done = False
        self.stop_tokens = ['färdig', 'slut', 'hejdå', 'done']  # TODO: Snyggare lösning
        self.max_replies = 2  # Maximum number of replies back and forth for each question
        self.nbr_replies = 0

        desc = 'InterviewWorld\t job: {}\t name: {}\t model: {}'.format(self.job, self.human_name, self.model_name)
        self.conversation_sv = BlenderConversation(lang='sv', tokenizer=self.tokenizer, description=desc)
        self.conversation_en = BlenderConversation(lang='en', tokenizer=self.tokenizer, description=desc)

        self.greet()

    def greet(self):
        print(self.greeting)
        return

    def start(self):
        # TODO: Prompt the user to add name and job they're looking for?
        return

    def reset_conversatoin(self):
        self.conversation_sv.reset()
        self.conversation_en.reset()
        return

    def chat(self, user_input):
        self.observe(user_input)
        self.act()
        return

    def observe(self, user_input):
        # TODO: Add spell check/grammar check here
        # Observe the user input, translate and update internal states
        # Check if user wants to quit/no questions left --> self.episode_done = True

        translated_input = self._sv_to_en(user_input)
        self.conversation_sv.add_user_text(user_input)
        self.conversation_en.add_user_text(translated_input)

        # Set episode done if exit conidion is met. TODO: Better check of input stop
        if self.nbr_replies == self.max_replies and len(self.questions) == 0 or user_input.lower().replace(' ',
                                                                                                           '') in self.stop_tokens:
            self.episode_done = True

        return

    def act(self):
        # Get context
        # Get

        if not self.episode_done:

            # fixa context
            # kör igneom model
            # strip token
            # addera output till convos
            # increment self.nbr_replies om modellsvar är ok, annars resetta till 0 och ta fråga från banken

            context = self._get_context()
            inputs = self._encode(context)
            set_trace()
            with no_grad():
                output_tokens = self.model.generate(**inputs)
            reply = self.tokenizer.decode(output_tokens[0],skip_special_tokens=True)

            if self._validate_reply(reply):
                self.nbr_replies += 1
            else:
                # TODO: More tries here? Change context or something
                reply = self.questions.pop()
                self.nbr_replies = 0

            translated_reply = self._en_to_sv(reply)
            self.conversation_sv.add_bot_text(translated_reply)
            self.conversation_en.add_bot_text(reply)
            self.conversation_sv.print_dialogue()

        else:
            self.conversation_sv.to_txt()
            self.conversation_en.to_txt()
            print('Tack för din intervju')
        return

    def _encode(self, text):
        return self.tokenizer([text], return_tensors='pt')

    def _decode(self, tokens):
        s = self.tokenizer.decode(tokens,skip_special_tokens=True)
        return s


    def _get_context(self):
        # Implement this in subclasses
        return self.conversation_en.get_dialogue_history()

    def _validate_reply(self, answer):
        # TODO:
        previous_replies = self.conversation_en.get_bot_replies()

        # If answer not in previous_replies ....
        if True:
            return True
        else:
            return False

    def _strip_token(self,line):
        # Removes SOS and EOS tokens from blenderbot reply
        line = line.replace('<s>', '')
        line = line.replace('</s>', '')
        return line

    # TODO: Change from googletrans to googles official API
    def _sv_to_en(self, text):
        out = self.translator.translate(text, src='sv', dest='en')
        return out.text

    def _en_to_sv(self, text):
        out = self.translator.translate(text, src='en', dest='sv')
        return out.text


def read_questions(file_path):
    # Reads interview questions from a text file, one question per line. '{}' in place where job should be inserted
    root = Path.cwd().parent
    file_path2 = root.joinpath('src/chat')
    file_path2 = file_path2.joinpath(file_path)
    with open(file_path2,'r') as f:
        questions = f.readlines()
    format_this = [True if '{}' in question else False for question in questions]
    return zip(questions,format_this)

In [18]:
name ='Alex'
job = 'data scientist'
mname = 'facebook/blenderbot-1B-distill' # options: 'facebook/blenderbot_small-90M' , 'facebook/blenderbot-400M-distill' ,'facebook/blenderbot-3B'
world = InterviewWorld(name=name, job=job,mname=mname)

Hej, och välkommen till din intervju. Hur står det till, Alex?


In [19]:
user_input = 'Tack det är bra, hur är det med dig?'
world.chat(user_input)

> <ipython-input-17-a95faca6262a>(87)act()
     85             inputs = self._encode(context)
     86             set_trace()
---> 87             with no_grad():
     88                 output_tokens = self.model.generate(**inputs)
     89             reply = self.tokenizer.decode(output_tokens[0],skip_special_tokens=True)

ipdb> n
> <ipython-input-17-a95faca6262a>(88)act()
     86             set_trace()
     87             with no_grad():
---> 88                 output_tokens = self.model.generate(**inputs)
     89             reply = self.tokenizer.decode(output_tokens[0],skip_special_tokens=True)
     90 

ipdb> n
> <ipython-input-17-a95faca6262a>(89)act()
     87             with no_grad():
     88                 output_tokens = self.model.generate(**inputs)
---> 89             reply = self.tokenizer.decode(output_tokens[0],skip_special_tokens=True)
     90 
     91             if self._validate_reply(reply):

ipdb> output_tokens?
*** SyntaxError: invalid syntax
ipdb> output_toke

BdbQuit: 

In [ ]:
output_tokens?